In [ ]:
# Third-party
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import h5py

from pyia import GaiaData

import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
import superfreq as sf

In [ ]:
gc_frame = coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.3*u.kpc)

In [ ]:
# see FGK-select.ipynb
g = GaiaData('../data/fgk.fits')
c = all_g.skycoord
galcen = c.transform_to(gc_frame)

In [ ]:
gal = c.galactic
gal.set_representation_cls('cartesian')

---

## Hexagons

In [ ]:
# internal triangle side length
h = 120. # pc

In [ ]:
def get_hexagons(h):
    a = np.sqrt(3)/2 * h # inner side
    
    pas = dict() # keyed by "ring"
    pas[0] = list()
    pas[1] = list()
    pas[2] = list()
#     pas[3] = list()
    
    pa0 = mpl.patches.RegularPolygon((0., 0.), numVertices=6, 
                                     radius=h, orientation=np.pi/2)
    pas[0].append(pa0.get_verts())

    for ang in np.arange(0, 360, 60)*u.deg:
        # Ring 1
        xy0 = 2*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[1].append(pa.get_verts())
        
        # Ring 2
        xy0 = 4*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
        xy0 = 3*h * np.array([np.cos(ang+120*u.deg), np.sin(ang+120*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
    return pas

In [ ]:
hexs = get_hexagons(h)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for k in hexs.keys():
    for pa in hexs[k]:
        pa = mpl.patches.Polygon(pa, facecolor='none', edgecolor='#333333')
        ax.add_patch(pa)

        
ax.plot(gal.u, gal.v, 
        marker=',', alpha=0.1, color='k',
        linestyle='none', zorder=100)
        
lim = 650
ax.set_xlim(-lim, lim)
ax.set_ylim(-lim, lim)

ax.set_xlabel('$x$ [pc]')
ax.set_ylabel('$y$ [pc]')

## Start with just inner hexagon:

In [ ]:
xyz = np.vstack((gal.u.to(u.pc).value, 
                 gal.v.to(u.pc).value,
                 gal.w.to(u.pc).value)).T

In [ ]:
pa = hexs[0][0]
hex_mask = mpl.patches.Path(pa).contains_points(xyz[:, :2])

In [ ]:
hex_mask.sum()

See `scripts/compute-freqs.py`:

In [ ]:
with h5py.File('../cache/freqs.hdf5') as f:
    freqs = f['freqs'][:]

In [ ]:
freq_qual_mask = ((freqs[:, 1] > 0.01) & # prograde
                  (freqs[:, 0] < 0.2) & (freqs[:, 1] < 0.2) & (freqs[:, 2] < 0.2))# reasonable values
z_mask = (np.abs(galcen.z) < 300*u.pc)

high_z_mask = (np.abs(galcen.z) > 300*u.pc)

In [ ]:
hex_freqs = np.abs(freqs[hex_mask & freq_qual_mask & freq_qual_mask])

In [ ]:
bins = np.linspace(0, 500, 128)
fbins = np.linspace(0.01, 0.12, 128)
fig, axes = plt.subplots(3, 2, figsize=(12, 8), sharex='col', sharey=True)

for i in range(3):
    axes[i, 0].hist(np.abs(hex_freqs[:, i]), bins=fbins);
    
    P = np.abs(2*np.pi / (hex_freqs[:, i] / u.Myr))
    axes[i, 1].hist(P[np.isfinite(P)], bins=bins);

axes[i, 0].set_xlim(0.01, 0.12)
axes[i, 1].set_xlim(0, 350)
    
fig.tight_layout()

In [ ]:
# bins = np.linspace(0.02, 0.12, 196)
# xlims = [(0.025, 0.07), None, (0.025, 0.07), (0.015, 0.05)]
# ylims = [(0.015, 0.044), None, (0.02, 0.12), (0.02, 0.12)]

# for k in hexs.keys():
#     for b, pa in enumerate(hexs[k]):
#         hex_mask = mpl.patches.Path(pa).contains_points(xyz[:, :2])
#         print(hex_mask.sum())
#         hex_freqs = np.abs(freqs[hex_mask & (freqs[:, 1] > 0.02) & # prograde
#                                  (freqs[:, 0] < 0.12) & (freqs[:, 1] < 0.12) & 
#                                  (freqs[:, 2] < 0.12)])
        
#         fig, axes = plt.subplots(2, 2, figsize=(8, 7.2))

#         for a, (i, j) in zip([0, 2, 3], [(0, 1), (0, 2), (1, 2)]):
#             ax = axes.flat[a]
#             H, xe, ye = np.histogram2d(hex_freqs[:,i], hex_freqs[:,j], bins=bins)
#             ax.pcolormesh(xe, ye, H.T, 
#                           norm=mpl.colors.LogNorm(), 
#                           cmap='magma', vmin=1, vmax=3E2)

#             ax.set_xlim(xlims[a])
#             ax.set_ylim(ylims[a])

#         axes[0, 0].set_ylabel(r'$\Omega_\phi$')
#         axes[1, 0].set_ylabel(r'$\Omega_z$')
#         axes[1, 0].set_xlabel(r'$\Omega_R$')
#         axes[1, 1].set_xlabel(r'$\Omega_\phi$')

#         # axes[0,1].set_visible(False)
#         ax = axes[0,1]
#         for k_ in hexs.keys():
#             for pa_ in hexs[k_]:
#                 pa_ = mpl.patches.Polygon(pa_, facecolor='none', edgecolor='#333333')
#                 ax.add_patch(pa_)

#         ax.add_patch(mpl.patches.Polygon(pa, facecolor='#333333', edgecolor='#333333'))
#         ax.set_xlim(-500, 500)
#         ax.set_ylim(-500, 500)
#         ax.set_xlabel('$x$ [pc]')
#         ax.set_ylabel('$y$ [pc]')

#         fig.tight_layout()

#         fig.savefig('../plots/freq/{0}-{1:02d}.png'.format(k, b), dpi=250)
#         plt.close(fig)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

ax = axes[0]
hex_freqs = np.abs(freqs[hex_mask & freq_qual_mask & z_mask])
ax.plot(hex_freqs[:, 2] / hex_freqs[:, 0],
        hex_freqs[:, 1] / hex_freqs[:, 0], 
        marker='.', linestyle='none', alpha=0.2)
ax.set_xlim(0.75, 2)
ax.set_ylim(0.65, 0.75)


ax = axes[1]
hex_freqs = np.abs(freqs[hex_mask & freq_qual_mask & high_z_mask])
ax.plot(hex_freqs[:, 2] / hex_freqs[:, 0],
        hex_freqs[:, 1] / hex_freqs[:, 0], 
        marker='.', linestyle='none', alpha=0.2)
ax.set_xlim(0.75, 2)
ax.set_ylim(0.65, 0.75)

fig.tight_layout()

In [ ]:
xbins = np.linspace(1.1, 2, 101)
ybins = np.linspace(0.69, 0.74, 101)

for k in hexs.keys():
    for b, pa in enumerate(hexs[k]):
        hex_mask = mpl.patches.Path(pa).contains_points(xyz[:, :2])
        hex_freqs = np.abs(freqs[hex_mask & freq_qual_mask & z_mask])
        print(hex_freqs.shape[0])
        
        fig, axes = plt.subplots(1, 2, figsize=(10, 4.8))

        xx = hex_freqs[:, 2] / hex_freqs[:, 0]
        yy = hex_freqs[:, 1] / hex_freqs[:, 0]

        H, xe, ye = np.histogram2d(xx, yy, bins=(xbins, ybins))
        ax = axes[0]
        ax.pcolormesh(xe, ye, H.T, 
                      norm=mpl.colors.LogNorm(), 
                      cmap='Greys', vmin=1, vmax=6e1)

        ax.set_xlabel(r'$\Omega_z / \Omega_R$')
        ax.set_ylabel(r'$\Omega_\phi / \Omega_R$')

        ax = axes[1]
        for k_ in hexs.keys():
            for pa_ in hexs[k_]:
                pa_ = mpl.patches.Polygon(pa_, facecolor='none', edgecolor='#333333')
                ax.add_patch(pa_)

        ax.add_patch(mpl.patches.Polygon(pa, facecolor='#333333', edgecolor='#333333'))
        ax.set_xlim(-520, 520)
        ax.set_ylim(-520, 520)
        ax.set_xlabel('$x$ [pc]')
        ax.set_ylabel('$y$ [pc]')
        ax.set_aspect('equal', 'datalim')
        
        fig.tight_layout()

        fig.savefig('../plots/freq/{0}-{1:02d}.png'.format(k, b), dpi=250)
        plt.close(fig)

#         break
#     break

```
convert -delay 30 -loop 0 1-*.png ring1.gif
convert -delay 30 -loop 0 2-*.png ring2.gif
convert -delay 30 -loop 1 *.png all.gif
```

In [ ]:
print('''convert -delay 30 -loop 0 1-*.png ring1.gif
convert -delay 30 -loop 0 2-*.png ring2.gif
convert -delay 30 -loop 1 *.png all.gif''')


line_cmd = 'convert -delay 30 -loop 0 {0} full-line{1}.gif'
for k, a in enumerate([[(2,0),(1,0),(0,0),(1,3),(2,6)],
                       [(2,2),(1,1),(0,0),(1,4),(2,8)],
                       [(2,4),(1,2),(0,0),(1,5),(2,10)]]):
    print(line_cmd.format(' '.join(['{0}-{1:02d}.png'.format(i, j) for i, j in a]), k))

## Bin in radius, azimuth?

In [ ]:
cyl = galcen.represent_as('cylindrical')

In [ ]:
Omp = (33.15*u.km/u.s/u.kpc).to(1/u.Myr).value
OmR_median = np.median(hex_freqs[:, 0])

In [ ]:
xbins = np.linspace(1.1, 2, 101)
ybins = np.linspace(0.69, 0.74, 101)

bins = np.arange(7.7, 8.8+1e-3, 0.1)
# bins = np.arange(8.1, 8.3+1e-3, 0.1)
# bins = np.arange(7.5, 7.7+1e-3, 0.1)

for k, l, r in zip(range(len(bins)), bins[:-1], bins[1:]):
    mask = (cyl.rho >= l*u.kpc) & (cyl.rho <= r*u.kpc)
    hex_freqs = np.abs(freqs[mask & freq_qual_mask & z_mask])
    N = hex_freqs.shape[0]
    print(N)

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))

    xx = hex_freqs[:, 2] / hex_freqs[:, 0]
    yy = hex_freqs[:, 1] / hex_freqs[:, 0]

    H, xe, ye = np.histogram2d(xx, yy, bins=(xbins, ybins))
    H = H * 2e5 / H.sum()
    ax.pcolormesh(xe, ye, H.T, 
                  norm=mpl.colors.LogNorm(), 
                  cmap='Greys', vmin=1, vmax=2e2)
    
#     for m in range(1, 15+1):
#         val = -1/m + Omp/OmR_median
#         ax.axhline(val, marker='')
        
#         if val < 0.74 and val > 0.69:
#             ax.text(1.11, val, '{0}:1'.format(m), fontsize=14)
            
#         print(-1/m + Omp/OmR_median)

    ax.set_title(r'${0:.1f} < R < {1:.1f}$'.format(l, r))

    ax.set_xlabel(r'$\Omega_z / \Omega_R$')
    ax.set_ylabel(r'$\Omega_\phi / \Omega_R$')
    ax.set_xlim(1.1, 2)
    ax.set_ylim(0.69, 0.74)

    fig.tight_layout()

    fig.savefig('../plots/freq/R-bin-{0}.png'.format(k), dpi=250)
    plt.close(fig)
    
fns = ['R-bin-{0}.png'.format(k) for k in range(len(bins)-1)]
fns = fns + fns[1:-1][::-1]
line_cmd = 'convert -delay 30 -loop 0 {0} R-anim.gif'.format(' '.join(fns))
print(line_cmd)

In [ ]:
fns = ['R-bin-{0}.png'.format(k) for k in range(len(bins)-1)]
fns = fns + fns[1:-1][::-1]
line_cmd = 'convert -delay 30 -loop 0 {0} R-anim.gif'.format(' '.join(fns))
print(line_cmd)

In [ ]:
xbins = np.linspace(1.1, 2, 101)
ybins = np.linspace(0.69, 0.74, 101)

bins = np.arange(175, 185+1e-3, 1)

for k, l, r in zip(range(len(bins)), bins[:-1], bins[1:]):
    mask = (cyl.phi.wrap_at(360*u.deg) >= l*u.deg) & (cyl.phi.wrap_at(360*u.deg) <= r*u.deg)
    hex_freqs = np.abs(freqs[mask & freq_qual_mask & z_mask])
    N = hex_freqs.shape[0]
    print(N)

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))

    xx = hex_freqs[:, 2] / hex_freqs[:, 0]
    yy = hex_freqs[:, 1] / hex_freqs[:, 0]

    H, xe, ye = np.histogram2d(xx, yy, bins=(xbins, ybins))
    H = H * 2e5 / H.sum()
    ax.pcolormesh(xe, ye, H.T, 
                  norm=mpl.colors.LogNorm(), 
                  cmap='Greys', vmin=1, vmax=2e2)

    ax.set_xlabel(r'$\Omega_z / \Omega_R$')
    ax.set_ylabel(r'$\Omega_\phi / \Omega_R$')
    ax.set_xlim(1.1, 2)
    ax.set_ylim(0.69, 0.74)

    fig.tight_layout()

    fig.savefig('../plots/freq/phi-bin-{0}.png'.format(k), dpi=250)
    plt.close(fig)